# POLICY

In [1]:
pip install -e .

Obtaining file:///C:/Users/Ernest/BITMEXMYSLAVE/bitmex-sandbox
  Found existing installation: bitmex-sandbox 0.0.1
    Uninstalling bitmex-sandbox-0.0.1:
      Successfully uninstalled bitmex-sandbox-0.0.1
  Running setup.py develop for bitmex-sandbox
Note: you may need to restart the kernel to use updated packages.


In [2]:
 # self.action_space = spaces.Discrete(2)
  #      self.observation_space = spaces.Box(-high, high, dtype=np.float32)

In [3]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
from IPython.display import clear_output
from numba import njit

In [4]:
import gym
import bitmex_sandbox
env = gym.make('bitbox-v0')
env.seed(1)
torch.manual_seed(1)
learning_rate = 0.01
gamma = 0.99

C:\ProgramData\Anaconda3\lib\site-packages\swagger_spec_validator\validator20.py:53: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  ref_dict['$ref'], '/'.join(path),


In [5]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        state_space = env.observation_space.shape[0]*env.observation_space.shape[1]
        action_space = env.action_space.n
        num_hidden = 128

        self.l1 = nn.Linear(state_space, num_hidden, bias=False)
        self.l2 = nn.Linear(num_hidden, action_space, bias=False)

        # Overall reward and loss history
        self.reward_history = []
        self.loss_history = []
        self.reset()

    def reset(self):
        # Episode policy and reward history
        self.episode_actions = torch.Tensor([])
        self.episode_rewards = []

    def forward(self, x):
        model = torch.nn.Sequential(
            self.l1,
            nn.Dropout(p = 0.5),
            nn.ReLU(),
            self.l2,
            nn.Softmax(dim=-1)
        )
        return model(x)

In [6]:
def predict(state):
    # Select an action (0 or 1) by running policy model
    # and choosing based on the probabilities in state
    state = torch.from_numpy(state).type(torch.FloatTensor)
    state = torch.reshape(state, (-1,))
    action_probs = policy(state)
    distribution = Categorical(action_probs)
    action = distribution.sample()
    # Add log probability of our chosen action to our history
    policy.episode_actions = torch.cat([
        policy.episode_actions,
        distribution.log_prob(action).reshape(1)
    ])
   
    return action

In [7]:
def update_policy():
    R = 0
    rewards = []
    # Discount future rewards back to the present using gamma
    for r in policy.episode_rewards[::-1]:
        R = r + gamma * R
        rewards.insert(0, R)
    # Scale rewards
    rewards = torch.FloatTensor(rewards)
    rewards = (rewards - rewards.mean()) / \
        (rewards.std() + np.finfo(np.float32).eps)

    # Calculate loss
    loss = (torch.sum(torch.mul(policy.episode_actions, rewards).mul(-1), -1))
    # Update network weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Save and intialize episode history counters
    policy.loss_history.append(loss.item())
    policy.reward_history.append(np.sum(policy.episode_rewards))
    policy.reset()
    return loss

In [8]:
def train(episodes, profits=[]):
    for episode in range(episodes):
        # Reset environment and record the starting state
        state = env.reset()
        for time in range(250):
            action = predict(state)
            # Uncomment to render the visual state in a window
            # env.render()
            # Step through environment using chosen action
            state, reward, done, _ = env.step(action.item())
            # Save reward
            policy.episode_rewards.append(reward)
            if done:
                break
        update_policy()
        profit = env._get_stat()
        profits.append(profit)
        # Calculate score to determine when the environment has been solved
        #profits.append(time)
        #if episode % 50 == 0:
        clear_output(wait=True)
        print('♂LASTEPISODE♂',episode,'/',episodes)
    return profits
        #print('♂LOSS♂',loss)
        
        #env.render()
        #if mean_score > env.spec.reward_threshold:
         #   print("Solved after {} episodes! Running average is now {}. Last episode ran to {} time steps."
          #        .format(episode, mean_score, time))
            #break

In [9]:
policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=learning_rate)
profits = train(episodes = 350)

♂LASTEPISODE♂ 349 / 350


In [10]:
df = pd.DataFrame(profits)

In [11]:
df.describe()

,0
count,350.000000
mean,-0.000322
std,0.006118
min,-0.026890
25%,-0.000675
50%,0.000661
75%,0.002757
max,0.009811


In [13]:
for _ in df[0]:
    print (_)

0.0012333841784565123
-0.003253454485172438
-0.010208659774854181
0.0004171587632477245
-0.003949654482006062
-0.0008662357369794895
0.0008672617427771274
0.0025440163124883317
-0.00133867393695708
0.0001804132711954471
6.207826361271686e-05
-0.0037274792240684553
-0.00046090973842101944
-0.003316978163696518
-0.002141153128791087
-0.004480238811467354
-0.01629287031757997
-0.01070034440216149
-0.0133062058799328
-0.004149417298892613
-0.010064852082038325
-0.013721125379585133
-0.012737227808727982
-0.01277795385559776
-0.01057153795984144
0.0009186569895041161
0.0009510826502261514
-0.019295486450928594
-0.013907517849165765
-0.011073782370584975
-0.005771438233698403
-0.00014458829096796431
-0.011342343748839903
-0.011274474640042711
0.035095330398682074
-0.004519417257969996
0.0034519176056233645
-0.015257349977988213
-0.010648643266999649
-0.0044446113290303926
-0.010980473414773364
-0.010336746911760111
0.07447697584011871
0.0335804397266184
-0.010974379916379054
-0.0116933759377

In [10]:
100  -0.000050
200  -0.000002
300   0.000015
350   0.0006
375 
400   0.000002

,0
0,5.346297e-08
1,-1.351913e-04
2,4.669230e-05
3,-2.476696e-04
4,-1.341093e-04
...,...
9995,7.101197e-04
9996,2.850708e-04
9997,3.917192e-04
9998,2.760885e-04


0.01 btc = 10к рубасов

In [ ]:
coef = 990643

In [9]:
500//60

8

In [14]:
policy.loss_history

[3.840394973754883,
 -0.2338792085647583,
 -24.416540145874023,
 19.141708374023438,
 49.581851959228516,
 -0.9400038719177246,
 -1.8194048404693604,
 -2.926384449005127,
 0.7324581146240234,
 6.731853008270264,
 -2.5922794342041016,
 -17.049484252929688,
 1.089423656463623,
 -11.756760597229004,
 7.011575222015381,
 -5.06822395324707,
 -16.588624954223633,
 9.560016632080078,
 -2.2037763595581055,
 -7.189038276672363,
 30.279682159423828,
 -7.612517356872559,
 -13.956741333007812,
 54.37318420410156,
 -17.36062240600586,
 -26.82618522644043,
 -3.0424981117248535,
 5.091420650482178,
 -4.021714687347412,
 -7.624608516693115,
 -0.0707709789276123,
 -7.011019229888916,
 2.4526588916778564,
 -6.6116108894348145,
 -0.02468550205230713,
 -2.53165340423584,
 1.2555439472198486,
 0.42546433210372925,
 2.413024425506592,
 -17.055339813232422,
 -15.370950698852539,
 0.8566834926605225,
 0.3255826532840729,
 4.424615383148193,
 4.455345630645752,
 -3.7158615589141846,
 3.6891260147094727,
 -0.07

# GYM

In [ ]:
# импорты
import gym, bitmex
from gym import spaces
from gym.utils import seeding   
import random, json
import pandas as pd
import numpy as np

In [ ]:
# константы
MAX_ACCOUNT_BALANCE = 2147483647
MAX_NUM_SHARES = 2147483647
MAX_SHARE_PRICE = 13000
MAX_STEPS = 20000
INITIAL_ACCOUNT_BALANCE = 1000000000
BIN_SIZE = '5m'
SYMBOL = 'XBTUSD'
OBSERV_WIN = 5

In [ ]:
class SandboxSnd(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self):
        super(SandboxSnd, self).__init__()

        self.df = self._get_df(BIN_SIZE,SYMBOL,MAX_STEPS)
        self.reward_range = (0, MAX_ACCOUNT_BALANCE)

        # Actions of the format Buy x%, Sell x%, Hold, etc.
        self.action_space = spaces.Box(
            low = np.array([0, 0]), high = np.array([3, 1]), dtype=np.float16)

        # Prices contains the OHCL values for the last five prices
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(6, 6), dtype=np.float16)
    def _get_df(self, binSize, symbol, MAX_STEPS):
        client = bitmex.bitmex(test=True)
        out = []
        for _ in range(0, MAX_STEPS, 1000):
            page = client.Trade.Trade_getBucketed(
                    binSize = binSize,
                    symbol = symbol,
                    count = 1000,
                    start = _ ,
                    reverse=True
                ).result()[0]
            out.extend(page)
        out.reverse()
        df = pd.DataFrame(out)
        return df
    def _next_observation(self):
        # Get the stock data points for the last 5 days and scale to between 0-1
        frame = np.array([
            self.df.loc[self.current_step: self.current_step +
                        OBSERV_WIN, 'open'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        OBSERV_WIN, 'high'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        OBSERV_WIN, 'low'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        OBSERV_WIN, 'close'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        OBSERV_WIN, 'volume'].values / MAX_NUM_SHARES,
        ])

        # Append additional data and scale each value to between 0-1
        obs = np.append(frame, [[
            self.balance / MAX_ACCOUNT_BALANCE,
            self.max_net_worth / MAX_ACCOUNT_BALANCE,
            self.shares_held / MAX_NUM_SHARES,
            self.cost_basis / MAX_SHARE_PRICE,
            self.total_shares_sold / MAX_NUM_SHARES,
            self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),
        ]], axis=0)

        return obs
#######################
    def _take_action(self, action):
        # Set the current price to a random price within the time step
        current_price = random.uniform(self.df.loc[self.current_step, "open"],
                                       self.df.loc[self.current_step, "close"])
        action_type = action[0]
        amount = action[1]

        if action_type < 1:
            # Buy amount % of balance in shares
            total_possible = int(self.balance / current_price)
            shares_bought = int(total_possible * amount)
            prev_cost = self.cost_basis * self.shares_held
            additional_cost = shares_bought * current_price

            self.balance -= additional_cost
            self.cost_basis = (prev_cost + additional_cost) / (self.shares_held + shares_bought)
            self.shares_held += shares_bought

        elif action_type < 2:
            # Sell amount % of shares held
            shares_sold = int(self.shares_held * amount)
            self.balance += shares_sold * current_price
            self.shares_held -= shares_sold
            self.total_shares_sold += shares_sold
            self.total_sales_value += shares_sold * current_price

        self.net_worth = self.balance + self.shares_held * current_price

        if self.net_worth > self.max_net_worth:
            self.max_net_worth = self.net_worth

        if self.shares_held == 0:
            self.cost_basis = 0

    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        if self.current_step > MAX_STEPS-OBSERV_WIN-1:
            self.current_step = 0

        delay_modifier = (self.current_step / MAX_STEPS)

        reward = self.balance * delay_modifier
        done = self.net_worth <= 0

        obs = self._next_observation()

        return obs, reward, done, {}

    def reset(self):
        # Reset the state of the environment to an initial state
        self.balance = INITIAL_ACCOUNT_BALANCE
        self.net_worth = INITIAL_ACCOUNT_BALANCE
        self.max_net_worth = INITIAL_ACCOUNT_BALANCE
        self.shares_held = 0
        self.cost_basis = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Set the current step to a random point within the data frame
        self.current_step = random.randint(0, MAX_STEPS-OBSERV_WIN-1)
        return self._next_observation()

    def render(self, mode='human', close=False):
        # Render the environment to the screen
        profit = self.net_worth - INITIAL_ACCOUNT_BALANCE

        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance}')
        print(
            f'Shares held: {self.shares_held} (Total sold: {self.total_shares_sold})')
        print(
            f'Avg cost for held shares: {self.cost_basis} (Total sales value: {self.total_sales_value})')
        print(
            f'Net worth: {self.net_worth} (Max net worth: {self.max_net_worth})')
        print(f'Profit: {profit}')

In [ ]:
#################
# импорты
import gym, bitmex
from gym import spaces
from gym.utils import seeding   
import random, json
import pandas as pd
import numpy as np
#################
# константы
MAX_ACCOUNT_BALANCE = 1
MAX_NUM_SHARES = 100000
MAX_SHARE_PRICE = 1/13000
MAX_STEPS = 20000
INITIAL_ACCOUNT_BALANCE = 0.01
BIN_SIZE = '5m'
SYMBOL = 'XBTUSD'
OBSERV_WIN = 5
#################
class SandboxSnd(gym.Env):
    metadata = {'render.modes': ['human']}
#################
    def __init__(self):
        super(SandboxSnd, self).__init__()

        self.df = self._get_df(BIN_SIZE,SYMBOL,MAX_STEPS)
        self.reward_range = (0, MAX_ACCOUNT_BALANCE)

        # Actions of the format Buy x%, Sell x%, Hold, etc.
        self.action_space = spaces.Box(
            low = np.array([0, 0]), high = np.array([3, 1]), dtype=np.float16)

        # Prices contains the OHCL values for the last five prices
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(6, 6), dtype=np.float16)
#################
    def _get_df(self, binSize, symbol, MAX_STEPS):
        client = bitmex.bitmex(test=True)
        out = []
        for _ in range(0, MAX_STEPS, 1000):
            page = client.Trade.Trade_getBucketed(
                    binSize = binSize,
                    symbol = symbol,
                    count = 1000,
                    start = _ ,
                    reverse=True
                ).result()[0]
            out.extend(page)
        out.reverse()
        df = pd.DataFrame(out)
        return df
 #################
    def _next_observation(self):
        # Get the stock data points for the last 5 days and scale to between 0-1
        frame = np.array([
            (1/self.df.loc[self.current_step: self.current_step +
                        OBSERV_WIN, 'open'].values) / MAX_SHARE_PRICE,
            (1/self.df.loc[self.current_step: self.current_step +
                        OBSERV_WIN, 'high'].values) / MAX_SHARE_PRICE,
            (1/self.df.loc[self.current_step: self.current_step +
                        OBSERV_WIN, 'low'].values) / MAX_SHARE_PRICE,
            (1/self.df.loc[self.current_step: self.current_step +
                        OBSERV_WIN, 'close'].values) / MAX_SHARE_PRICE,
            (1/self.df.loc[self.current_step: self.current_step +
                        OBSERV_WIN, 'volume'].values) / MAX_NUM_SHARES,
        ])

        # Append additional data and scale each value to between 0-1
        obs = np.append(frame, [[
            self.balance / MAX_ACCOUNT_BALANCE,
            self.max_net_worth / MAX_ACCOUNT_BALANCE,
            self.shares_held / MAX_NUM_SHARES,
            self.cost_basis / MAX_SHARE_PRICE,
            self.total_shares_sold / MAX_NUM_SHARES,
            self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),
        ]], axis=0)

        return obs
#######################
    def _take_action(self, action):
        # Set the current price to a random price within the time step
        current_price = 1/random.uniform(self.df.loc[self.current_step,"open"],
                                         self.df.loc[self.current_step, "close"])
        action_type = action[0]
        amount = action[1]
        total_possible = int(self.balance / current_price)
        shares_val = int(total_possible * amount)
        if action_type < 1:
            # Buy amount % of balance in shares
            prev_cost = self.cost_basis * self.shares_held
            additional_cost = shares_val * current_price
            self.balance -= additional_cost
            self.cost_basis = (prev_cost + additional_cost) / (self.shares_held + shares_val)
            self.shares_held += shares_val

        elif action_type < 2:
            # Sell amount % of shares held
            self.balance += shares_val * current_price
            self.shares_held -= shares_val
            self.total_shares_sold += shares_val
            self.total_sales_value += shares_val * current_price

        self.net_worth = self.balance + self.shares_held * current_price

        if self.net_worth > self.max_net_worth:
            self.max_net_worth = self.net_worth

        if self.shares_held == 0:
            self.cost_basis = 0
#################
    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        if self.current_step > MAX_STEPS-OBSERV_WIN-1:
            self.current_step = 0

        delay_modifier = (self.current_step / MAX_STEPS)

        reward = self.net_worth * delay_modifier
        done = self.net_worth <= 0

        obs = self._next_observation()

        return obs, reward, done, {}
#################
    def reset(self):
        # Reset the state of the environment to an initial state
        self.balance = INITIAL_ACCOUNT_BALANCE
        self.net_worth = INITIAL_ACCOUNT_BALANCE
        self.max_net_worth = INITIAL_ACCOUNT_BALANCE
        self.shares_held = 0
        self.cost_basis = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Set the current step to a random point within the data frame
        self.current_step = random.randint(0, MAX_STEPS-OBSERV_WIN-1)
        return self._next_observation()
#################
    def render(self, mode='human', close=False):
        # Render the environment to the screen
        profit = self.net_worth - INITIAL_ACCOUNT_BALANCE

        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance}')
        print(
            f'Shares held: {self.shares_held} (Total sold: {self.total_shares_sold})')
        print(
            f'Avg cost for held shares: {self.cost_basis} (Total sales value: {self.total_sales_value})')
        print(
            f'Net worth: {self.net_worth} (Max net worth: {self.max_net_worth})')
        print(f'Profit: {profit}')

# test

In [ ]:
import random
import json
import gym
from gym import spaces
import numpy as np
import pandas as pd
import bitmex


client = bitmex.bitmex(test=True)
out = []
for i in range(0, 5000, 1000):
    page = client.Trade.Trade_getBucketed(
            binSize = '1m',
            symbol = 'XBTUSD',
            count = 1000,
            start = i,
            reverse=True
        ).result()[0]
    out.extend(page)
out.reverse()
df = pd.DataFrame(out)


MAX_ACCOUNT_BALANCE = 2147483647 # максимальный баланс
MAX_NUM_SHARES = 2147483647 # максимальное число контрактов
MAX_SHARE_PRICE = 5000 # максимальная цена контракта
MAX_OPEN_POSITIONS = 5 # максимальное число открытых позиций
MAX_STEPS = 20000 # число итераций в эпизоде

INITIAL_ACCOUNT_BALANCE = 100000000000000 # баланс в начале


class SandboxSnd(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self):
        super(SandboxSnd, self).__init__()

        self.df = df
        self.reward_range = (0, MAX_ACCOUNT_BALANCE)

        # Actions of the format Buy x%, Sell x%, Hold, etc.
        self.action_space = spaces.Box(low = np.array([0, 0]),
                                       high = np.array([3, 1]), dtype=np.float16)
        # Prices contains the OHCL values for the last five prices
        self.observation_space = spaces.Box(low=0, high=1, shape=(6, 6),
                                            dtype=np.float16)
    def _next_observation(self):
        # Get the stock data points for the last 5 days and scale to between 0-1
        frame = np.array([
            self.df.loc[self.current_step: self.current_step +
                        5, 'open'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'high'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'low'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'close'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'volume'].values / MAX_NUM_SHARES])

        # Append additional data and scale each value to between 0-1
        obs = np.append(frame, [[
            self.balance / MAX_ACCOUNT_BALANCE,
            self.max_net_worth / MAX_ACCOUNT_BALANCE,
            self.shares_held / MAX_NUM_SHARES,
            self.cost_basis / MAX_SHARE_PRICE,
            self.total_shares_sold / MAX_NUM_SHARES,
            self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),
        ]], axis=0)

        return obs

    def _take_action(self, action):
        # Set the current price to a random price within the time step
        current_price = random.uniform(
            self.df.loc[self.current_step, "open"], self.df.loc[self.current_step, "close"])

        action_type = action[0]
        amount = action[1]

        if action_type < 1:
            # Buy amount % of balance in shares
            total_possible = int(self.balance / current_price)
            shares_bought = int(total_possible * amount)
            prev_cost = self.cost_basis * self.shares_held
            additional_cost = shares_bought * current_price

            self.balance -= additional_cost
            self.cost_basis = (
                prev_cost + additional_cost) / (self.shares_held + shares_bought)
            self.shares_held += shares_bought

        elif action_type < 2:
            # Sell amount % of shares held
            shares_sold = int(self.shares_held * amount)
            self.balance += shares_sold * current_price
            self.shares_held -= shares_sold
            self.total_shares_sold += shares_sold
            self.total_sales_value += shares_sold * current_price

        self.net_worth = self.balance + self.shares_held * current_price

        if self.net_worth > self.max_net_worth:
            self.max_net_worth = self.net_worth

        if self.shares_held == 0:
            self.cost_basis = 0

    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        if self.current_step > len(self.df.loc[:, 'open'].values) - 6:
            self.current_step = 0

        delay_modifier = (self.current_step / MAX_STEPS)

        reward = self.balance * delay_modifier
        done = self.net_worth <= 0

        obs = self._next_observation()

        return obs, reward, done, {}

    def reset(self):
        # Reset the state of the environment to an initial state
        self.balance = INITIAL_ACCOUNT_BALANCE
        self.net_worth = INITIAL_ACCOUNT_BALANCE
        self.max_net_worth = INITIAL_ACCOUNT_BALANCE
        self.shares_held = 0
        self.cost_basis = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Set the current step to a random point within the data frame
        self.current_step = random.randint(
            0, len(self.df.loc[:, 'open'].values) - 6)
        return self._next_observation()

    def render(self, mode='human', close=False):
        # Render the environment to the screen
        profit = self.net_worth - INITIAL_ACCOUNT_BALANCE

        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance}')
        print(
            f'Shares held: {self.shares_held} (Total sold: {self.total_shares_sold})')
        print(
            f'Avg cost for held shares: {self.cost_basis} (Total sales value: {self.total_sales_value})')
        print(
            f'Net worth: {self.net_worth} (Max net worth: {self.max_net_worth})')
        print(f'Profit: {profit}')

In [ ]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding

#class SandboxSnd(gym.Env):
 #   metadata = {'render.modes': ['human']}
  #  def __init__(self):
   #     pass
    #def step(self, action):
     #   pass
    #def reset(self):
     #   pass
    #def render(self, mode='human', close=False):
     #   pass
    
    
import random
import json
import gym
from gym import spaces
import pandas as pd
import numpy as np

import pandas as pd
import bitmex

client = bitmex.bitmex(test=True)
out = []
for i in range(0, 5000, 1000):
    page = client.Trade.Trade_getBucketed(
            binSize = '1m',
            symbol = 'XBTUSD',
            count = 1000,
            start = i,
            reverse=True
        ).result()[0]
    out.extend(page)
out.reverse()
df = pd.DataFrame(out)

MAX_ACCOUNT_BALANCE = 2147483647
MAX_NUM_SHARES = 2147483647
MAX_SHARE_PRICE = 5000
MAX_OPEN_POSITIONS = 5
MAX_STEPS = 20000

INITIAL_ACCOUNT_BALANCE = 100000000000000


class SandboxSnd(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self):
        super(SandboxSnd, self).__init__()

        self.df = df
        self.reward_range = (0, MAX_ACCOUNT_BALANCE)

        # Actions of the format Buy x%, Sell x%, Hold, etc.
        self.action_space = spaces.Box(
            low=np.array([0, 0]), high=np.array([3, 1]), dtype=np.float16)

        # Prices contains the OHCL values for the last five prices
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(6, 6), dtype=np.float16)

    def _next_observation(self):
        # Get the stock data points for the last 5 days and scale to between 0-1
        frame = np.array([
            self.df.loc[self.current_step: self.current_step +
                        5, 'open'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'high'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'low'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'close'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'volume'].values / MAX_NUM_SHARES,
        ])

        # Append additional data and scale each value to between 0-1
        obs = np.append(frame, [[
            self.balance / MAX_ACCOUNT_BALANCE,
            self.max_net_worth / MAX_ACCOUNT_BALANCE,
            self.shares_held / MAX_NUM_SHARES,
            self.cost_basis / MAX_SHARE_PRICE,
            self.total_shares_sold / MAX_NUM_SHARES,
            self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),
        ]], axis=0)

        return obs

    def _take_action(self, action):
        # Set the current price to a random price within the time step
        current_price = random.uniform(
            self.df.loc[self.current_step, "open"], self.df.loc[self.current_step, "close"])

        action_type = action[0]
        amount = action[1]

        if action_type < 1:
            # Buy amount % of balance in shares
            total_possible = int(self.balance / current_price)
            shares_bought = int(total_possible * amount)
            prev_cost = self.cost_basis * self.shares_held
            additional_cost = shares_bought * current_price

            self.balance -= additional_cost
            self.cost_basis = (
                prev_cost + additional_cost) / (self.shares_held + shares_bought)
            self.shares_held += shares_bought

        elif action_type < 2:
            # Sell amount % of shares held
            shares_sold = int(self.shares_held * amount)
            self.balance += shares_sold * current_price
            self.shares_held -= shares_sold
            self.total_shares_sold += shares_sold
            self.total_sales_value += shares_sold * current_price

        self.net_worth = self.balance + self.shares_held * current_price

        if self.net_worth > self.max_net_worth:
            self.max_net_worth = self.net_worth

        if self.shares_held == 0:
            self.cost_basis = 0

    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        if self.current_step > len(self.df.loc[:, 'open'].values) - 6:
            self.current_step = 0

        delay_modifier = (self.current_step / MAX_STEPS)

        reward = self.balance * delay_modifier
        done = self.net_worth <= 0

        obs = self._next_observation()

        return obs, reward, done, {}

    def reset(self):
        # Reset the state of the environment to an initial state
        self.balance = INITIAL_ACCOUNT_BALANCE
        self.net_worth = INITIAL_ACCOUNT_BALANCE
        self.max_net_worth = INITIAL_ACCOUNT_BALANCE
        self.shares_held = 0
        self.cost_basis = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Set the current step to a random point within the data frame
        self.current_step = random.randint(
            0, len(self.df.loc[:, 'open'].values) - 6)
        return self._next_observation()

    def render(self, mode='human', close=False):
        # Render the environment to the screen
        profit = self.net_worth - INITIAL_ACCOUNT_BALANCE

        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance}')
        print(
            f'Shares held: {self.shares_held} (Total sold: {self.total_shares_sold})')
        print(
            f'Avg cost for held shares: {self.cost_basis} (Total sales value: {self.total_sales_value})')
        print(
            f'Net worth: {self.net_worth} (Max net worth: {self.max_net_worth})')
        print(f'Profit: {profit}')